### Get environment name from widget and assign to variable 

In [0]:
dbutils.widgets.text(name="env", defaultValue='', label='Enter the environment in lower case')
env = dbutils.widgets.get("env")


<br>

### Call common notebook to access shared variables and methods

In [0]:
%run "./3_Common"


### Reading the data from bronze raw_traffic Table

In [0]:

def read_BronzeTrafficTable(environment):
    print('Reading the Bronze Table Data : ',end='')
    df_bronzeTraffic = (spark.readStream
                    .table(f"`{environment}_catalog`.`bronze`.raw_traffic")
                    )
    print(f'Reading {environment}_catalog.bronze.raw_traffic Success!')
    return df_bronzeTraffic


### Getting count of Electric vehicles by creating new column


In [0]:
def ev_Count(df):

    print('Creating Electric Vehicles Count Column : ', end='')

    from pyspark.sql.functions import col

    df_ev = df.withColumn('Electric_Vehicles_Count', 
                          col('EV_Car') + col('EV_Bike')
                          ) 
    
    print('Success!! ')

    return df_ev


### Creating columns to get Count of all motor vehicles


In [0]:
def Motor_Count(df):

    print('Creating All Motor Vehicles Count Column : ', end='')

    from pyspark.sql.functions import col

    df_motor = df.withColumn('Motor_Vehicles_Count',
                            col('Electric_Vehicles_Count') + col('Two_wheeled_motor_vehicles') + col('Cars_and_taxis') + col('Buses_and_coaches') + col('LGV_Type') + col('HGV_Type')
                            )
    
    print('Success!! ')
    
    return df_motor


### Creating Transformed Time column

In [0]:
def create_TransformedTime(df):

    from pyspark.sql.functions import current_timestamp

    print('Creating Transformed Time column : ', end='')

    df_timestamp = df.withColumn('Transformed_Time',
                      current_timestamp()
                      )
    
    print('Success!!')
    
    return df_timestamp


### Writing the Transformed data to Silver_Traffic Table

In [0]:
def write_Traffic_SilverTable(StreamingDF,environment):

    print('Writing the silver_traffic Data : ', end='') 

    write_StreamSilver = (StreamingDF.writeStream
                .format('delta')
                .option('checkpointLocation', checkpoint + "/SilverTrafficLoad/Checkpt/")
                .outputMode('append')
                .queryName("SilverTrafficWriteStream")
                .trigger(availableNow=True)
                .toTable(f"`{environment}_catalog`.`silver`.`silver_traffic`"))
    
    write_StreamSilver.awaitTermination()
    
    print(f'Writing `{environment}_catalog`.`silver`.`silver_traffic` Success!')

### Before adding new data

In [0]:
# Check if the table exists
tableExists = spark._jsparkSession.catalog().tableExists(f"{env}_catalog.silver.silver_traffic")

if tableExists:
    # If the table exists, execute the query
    query = f"SELECT count(*) as `Num of Rows` FROM `{env}_catalog`.`silver`.`silver_traffic`"
    df = spark.sql(query)
    display(df)
else:
    print("Table does not exist.")

## Calling all the functions

In [0]:
## Reading the bronze traffic data
df_trafficdata = read_BronzeTrafficTable(env)

# To remove duplicate rows
df_noDups = remove_Dups(df_trafficdata)

# Get dataframe columns name to a list
Allcolumns = df_noDups.schema.names

# Replace any NULL values in strings and numeric datatypes
df_noNulls = handle_NULLs(df_noDups, Allcolumns)

## To get the total EV_Count
df_ev = ev_Count(df_noNulls)

## To get the Total Motor vehicle count
df_motor = Motor_Count(df_ev)

## Calling Transformed time function
df_final = create_TransformedTime(df_motor)

## Writing to silver_traffic
write_Traffic_SilverTable(df_final, env)

### After adding new data

In [0]:
 %sql

SELECT count(*) as `Num of Rows` FROM `dev_catalog`.`silver`.`silver_traffic`

### Display small sample of data

In [0]:
%sql

SELECT * FROM `dev_catalog`.`silver`.`silver_traffic` Limit 5